<a href="https://colab.research.google.com/github/Harikrishna035/Harikrishna.github.io/blob/main/Diabetic_retinopathy_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"byaswanthreddy","key":"4b43720c9ee4d524efdee6888ea27b83"}'}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c aptos2019-blindness-detection

  0% 2.00M/9.51G [00:01<1:16:10, 2.23MB/s]

In [ ]:
!unzip aptos2019-blindness-detection.zip

In [ ]:
!rm aptos2019-blindness-detection.zip

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR

In [ ]:
df=pd.read_csv('train.csv')
df.head()

In [ ]:
imgs_path='train_images/'
img_size=128
class_names=['No DR','Mild','Moderate','Severe','Proliferative DR']
class_names

['No DR', 'Mild', 'Moderate', 'Severe', 'Proliferative DR']

In [ ]:
labels=df['diagnosis']
labels

In [ ]:
def get_data(data_dir):
    img_data = []

    for img in sorted(os.listdir(data_dir)):
            try:
                img_arr = cv2.imread(os.path.join(data_dir, img), cv2.IMREAD_COLOR)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                img_data.append(resized_arr)
                # labels.append(class_num)
            except Exception as e:
                print(e)
    return np.array(img_data)

In [ ]:
img_data= get_data(imgs_path)
labels=np.array(labels)
print(f'the shape of input image data is {img_data.shape}, labels shape is {labels.shape}')

the shape of input image data is (3662, 128, 128, 3), labels shape is (3662,)


In [ ]:
import random
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=random.randint(0,len(img_data))
        ax[i,j].imshow(img_data[l])
        ax[i,j].set_title('output class is : '+str(class_names[labels[l]]))

plt.tight_layout()

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
img_data=preprocess_input(img_data)
img_data[0]

In [ ]:
from tensorflow.keras.utils import to_categorical
labels=to_categorical(labels)
labels[0]

array([0., 0., 1., 0., 0.], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(img_data,labels,test_size=0.2,random_state=0,stratify=labels)
print(f'X_train size is {X_train.shape}, X_test shape is {X_test.shape}')

X_train size is (2929, 128, 128, 3), X_test shape is (733, 128, 128, 3)


In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# x_train=preprocess_input(x_train)
inception=VGG19(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
for layer in inception.layers[:-1]:
  layer.trainable=False

model=Sequential()
model.add(inception)
model.add(Flatten())
model.add(Dense(len(class_names),activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
learning_rate=ReduceLROnPlateau(monitor='val_accuracy',
                                           patience=3,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr=0.0001)
cb_early_stop=EarlyStopping(monitor = 'val_loss', patience = 2)
callbacks_list=[learning_rate,cb_early_stop]

In [ ]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7,batch_size=64,callbacks=[callbacks_list])

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=2.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=2.0)
plt.legend(['Training accuracy', 'Validation accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('accuracy',fontsize=16)
plt.title('val accuracy',fontsize=16)

In [ ]:
y_pred=model.predict(X_test)
y_pred_classes=np.argmax(y_pred,axis=1)
y_test_labels=np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_recall_fscore_support
# print(confusion_matrix(y_test_labels,y_pred_classes))
sns.heatmap(confusion_matrix(y_test_labels,y_pred_classes),yticklabels=class_names,xticklabels=class_names,annot=True,fmt='.1f') # calculating the confusion matrix
accuracy=accuracy_score(y_test_labels,y_pred_classes)*100 # calculating the accuracy
print(f'Accuracy : {accuracy}')
prec=precision_recall_fscore_support(y_test_labels,y_pred_classes)
print('precision is ',prec[0].mean())
print('Recall is ',prec[1].mean())
print('f1 score is ',prec[2].mean())
print('support is ',int(prec[-1].mean()))

plt.show()

In [ ]:
# precision recall curve
from sklearn.metrics import precision_recall_curve
precision = dict()
recall = dict()
for i in range(len(class_names)):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_pred[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(class_names[i]))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# x_train=preprocess_input(x_train)
inception=InceptionV3(weights='imagenet',include_top=False,input_shape=(img_size,img_size,3))
for layer in inception.layers[:-1]:
  layer.trainable=False

model2=Sequential()
model2.add(inception)
model2.add(Flatten())
model2.add(Dense(len(class_names),activation='softmax'))

model2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model2.summary()

In [ ]:
history2=model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7,batch_size=64,callbacks=[callbacks_list])

In [ ]:
plt.figure(figsize=[8,6])
plt.plot(history2.history['accuracy'],'r',linewidth=2.0)
plt.plot(history2.history['val_accuracy'],'b',linewidth=2.0)
plt.legend(['Training accuracy', 'Validation accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('accuracy',fontsize=16)
plt.title('val accuracy',fontsize=16)

In [ ]:
y_pred2=model.predict(X_test)
y_pred_classes2=np.argmax(y_pred2,axis=1)
y_test_labels2=np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_recall_fscore_support
# print(confusion_matrix(y_test_labels,y_pred_classes))
sns.heatmap(confusion_matrix(y_test_labels2,y_pred_classes2),yticklabels=class_names,xticklabels=class_names,annot=True,fmt='.1f') # calculating the confusion matrix
accuracy2=accuracy_score(y_test_labels2,y_pred_classes2)*100 # calculating the accuracy
print(f'Accuracy : {accuracy2}')
prec=precision_recall_fscore_support(y_test_labels2,y_pred_classes2)
print('precision is ',prec[0].mean())
print('Recall is ',prec[1].mean())
print('f1 score is ',prec[2].mean())
print('support is ',int(prec[-1].mean()))

In [ ]:
# precision recall curve
from sklearn.metrics import precision_recall_curve
precision = dict()
recall = dict()
for i in range(len(class_names)):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_pred2[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(class_names[i]))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()

In [ ]:
results=pd.DataFrame(data=[[accuracy,accuracy2]],columns=['VGG19','InceptionV3'])
# results.head()
sns.barplot(data=results)
plt.title('comparision of model accuracies')
plt.xlabel('model')
plt.ylim([0,100])
plt.ylabel('accuracy')

In [ ]:
files.upload() # upload image for test

In [ ]:
image_name='/content/test_images/00836aaacf06.png'   # enter image name to test

x1=[]
img_1 = cv2.imread(image_name, cv2.IMREAD_COLOR)
resized_arr_1 = cv2.resize(img_1, (img_size, img_size)) # Reshaping images to preferred size
plt.imshow(resized_arr_1)
x1.append([resized_arr_1])
x1=np.array(x1)
x1 = preprocess_input(x1)
x1 = x1.reshape(-1, img_size, img_size, 3)
x1.shape

In [ ]:
pred=model.predict(x1) # for predicting class
labels_pred=np.argmax(pred,axis=1)
labels=class_names[labels_pred[0]]
pred_results=pd.DataFrame(data=pred,columns=class_names)
fig=plt.figure(figsize=(10,8))
sns.set_theme(style="darkgrid")
ax=sns.barplot(data=pred_results)
ax.set_xticklabels(class_names,rotation=90)
ax.set_xlabel('Class')
ax.set_ylabel('Accuracy')
ax.set_title('Predicting class name ')
plt.show()